In [13]:
EXP_NAME = 'pretraining_XLMR_52'
FOLDER_NAME = EXP_NAME

In [14]:
import torch
torch.cuda.is_available()

True

In [15]:
import os

import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from datasets import Dataset
from sklearn import model_selection 
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

from tqdm.autonotebook import tqdm

import gc

import re
import random
random.seed(42)

import warnings
warnings.filterwarnings("ignore")

In [16]:
model_checkpoint = 'xlm-roberta-large'
batch_size = 8
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
max_length = 384
doc_stride = 192
pad_on_right = tokenizer.padding_side == "right"
n_folds = 5
n_epochs = 1
N_RANDOM = 2
N_ANSWER = 1
PAD = 32
MAX_LENGTH = 384

In [17]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [18]:
train = pd.read_csv("train.csv")
# test = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
# external_mlqa = pd.read_csv("../input/mlqa-hindi-processed/mlqa_hindi.csv")
# external_xquad = pd.read_csv("../input/mlqa-hindi-processed/xquad.csv")
# external_tydiqa = pd.read_csv("../input/tydi-qa-download/tydiqa_train.csv")
# external_train = pd.concat([external_mlqa, external_xquad, external_tydiqa])

# train = train.sample(n=10).reset_index(drop=True)

def create_folds(data, num_splits):
    data["kfold"] = -1
    kf = model_selection.StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=2021)
    for fold_num, (t_, v_) in enumerate(kf.split(X=data, y=data.language.values)):
        data.loc[v_, "kfold"] = fold_num
    return data

train = create_folds(train, num_splits=n_folds)
# external_train["kfold"] = -1
# external_train["id"] = list(np.arange(1, len(external_train) + 1))
# train = pd.concat([train, external_train]).reset_index(drop=True)


def convert_answers(row):
    return {"answer_start": [row[0]], "text": [row[1]]}

train["answers"] = train[["answer_start", "answer_text"]].apply(convert_answers, axis=1)

# del external_mlqa
# del external_xquad
# del external_tydiqa
# del external_train
# gc.collect();

In [19]:
train.head(1)

,id,context,question,answer_text,answer_start,language,kfold,answers
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,3,"{'answer_start': [53], 'text': ['206']}"


In [20]:
from datasets import load_dataset
squad = load_dataset("squad")

Reusing dataset squad (/home/darek/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
tydi = pd.read_csv('tydiqa_train.csv.zip')
tydi_bete_all = tydi[(tydi.language == 'bengali') | (tydi.language == 'telugu') | (tydi.language == 'english')].reset_index(drop=True)
print(len(tydi_bete_all))
tydi_bete_all.head(1)

11650


,id,context,question,answer_text,answer_start,language
0,6553242570085056655,"\nవేపచెట్టు, వేపాకు, వేపపూత ఇలా వేపచెట్టునుండి...",వేప చెట్టు యొక్క శాస్త్రీయ నామం ఏమిటి?,Azadirachta indica,702,telugu


In [22]:
threeds = pd.read_csv('chaii-mlqa-xquad-5folds.csv')
chaii = threeds[threeds.src == 'chaii'].reset_index(drop=True)
chaii = chaii.sample(n=300)
print(len(chaii))

chaii["answers"] = chaii[["answer_start", "answer_text"]].apply(convert_answers, axis=1)
tydi_bete_all["answers"] = tydi_bete_all[["answer_start", "answer_text"]].apply(convert_answers, axis=1)

chaii_ds = Dataset.from_pandas(chaii)
tydi_bete_all_ds = Dataset.from_pandas(tydi_bete_all)

tokenized_tydi_bete_all_strid = tydi_bete_all_ds.map(prepare_train_features, batched=True, remove_columns=tydi_bete_all_ds.column_names)
tokenized_chaii_strid = chaii_ds.map(prepare_train_features, batched=True, remove_columns=chaii_ds.column_names)


300


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
def negative_sampling(examples, ratio=0.1):
    
    def _sample(pos):
        if pos != 0: return True
        else: return random.random() < ratio
                
    indices = [i for i,x in enumerate(examples['start_positions']) if _sample(x)]

    for key in examples.keys():
        examples[key] = [x for i,x in enumerate(examples[key]) if i in indices]
        
    return examples

In [24]:
tokenized_tydi_bete_all_strid = tokenized_tydi_bete_all_strid.map(negative_sampling, batched=True, batch_size=8)

  0%|          | 0/41161 [00:00<?, ?ba/s]

In [25]:
tokenized_squad = squad['train'].map(prepare_train_features, batched=True, remove_columns=squad['train'].column_names)

# print((len(tokenized_tydi_bete_all_strid), len(tokenized_squad), len(tokenized_chaii_strid)))

  0%|          | 0/88 [00:00<?, ?ba/s]

In [26]:
from datasets import concatenate_datasets

tokenized_train_all = concatenate_datasets([tokenized_tydi_bete_all_strid, tokenized_squad])
tokenized_train_all = tokenized_train_all.shuffle(seed=1)

In [29]:
len(tokenized_train_all)

138118

In [27]:
fold = 0

In [28]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream t

In [30]:
args = TrainingArguments(
    f"chaii-qa-{EXP_NAME}",
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 500,
    save_strategy = "epoch",
    learning_rate=1e-5,
    gradient_accumulation_steps=4,
    warmup_ratio=0.2,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to='none',
    save_total_limit=15
)

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_all,
    eval_dataset=tokenized_chaii_strid,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [31]:
trainer.train()

***** Running training *****
  Num examples = 138118
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 8632


Step,Training Loss,Validation Loss
500,4.589900,0.678744
1000,1.381100,0.515704
1500,0.980900,0.524007
2000,0.872300,0.314025
2500,0.817500,0.504671
3000,0.775500,0.420500
3500,0.741800,0.356077
4000,0.738700,0.323773
4500,0.659100,0.448630
5000,0.602600,0.454255


***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
***** Running Evaluation *****
  Num examples = 4752
  Batch size = 8
Saving model checkpoint to chaii-qa-pretraining_XLMR_04/checkpoint-4316
Configuration saved in chaii-qa-pretraining_XLMR_04/checkpoint-4316/config.json
Model weights saved in chaii-qa-pretraining_XLMR_04/checkpoint-4316/pytorch_model.bin
tokenizer config file saved in chaii-qa-pretraining_XLMR_04/checkpoint-4316/tokenizer_config.json
Special tokens file saved in chaii-qa-pretraining_XLMR_04/checkpoint-4316/special_tokens_map.json
***** 

In [32]:
# !cp -a chaii-qa/checkpoint-9224 /content/drive/MyDrive/chaii/exp_021-XLMR-TYBETE/

In [33]:
# ls chaii-qa/checkpoint-9224

In [34]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [35]:
valid_dataset = Dataset.from_pandas(train)

validation_features = valid_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=valid_dataset.column_names
)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [36]:
valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

  0%|          | 0/19513 [00:00<?, ?ex/s]

In [37]:
raw_predictions = trainer.predict(valid_feats_small)

***** Running Prediction *****
  Num examples = 19513
  Batch size = 8


In [38]:
max_answer_length = 30

In [39]:
import collections

examples = valid_dataset
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [40]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [41]:
final_predictions = postprocess_qa_predictions(valid_dataset, validation_features, raw_predictions.predictions)

Post-processing 1114 example predictions split into 19513 features.


  0%|          | 0/1114 [00:00<?, ?it/s]

In [42]:
references = [{"id": ex["id"], "context": ex["context"], "question": ex["question"], "answer": ex["answers"]['text'][0]} for ex in valid_dataset]

In [43]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [44]:
res = pd.DataFrame(references)
res['prediction'] = res['id'].apply(lambda r: final_predictions[r])
res['jaccard'] = res[['answer', 'prediction']].apply(jaccard, axis=1)
res

,id,context,question,answer,prediction,jaccard
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,206,1.0
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,காசுமீரில்,1.0
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,அலெக் ஸாண்டர் ஃப்பௌமிங்,0.0
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,தாலாட்டு,1.0
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,சூரியன்,0.0
...,...,...,...,...,...,...
1109,26f356026,स्वामी निगमानन्द परमहंस (18 अगस्त 1880 - 29 नव...,स्वामी निगमानन्द परमहंस के तन्त्र गुरु कौन थे?,बामाक्षेपा,उदासिनाचार्य सुमेरुदास महाराज,0.0
1110,31179f1bb,भरत मुनि ने नाट्यशास्त्र नामक प्रसिद्ध ग्रन्थ ...,नित्यशास्त्र किसने लिखा है?,भरत मुनि,भरत मुनि,1.0
1111,0d35dc007,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,17,1.0
1112,7f997884d,"जलाल उद्दीन मोहम्मद अकबर () (१५ अक्तूबर, १५४२-...",मुगल सम्राट अकबर की मृत्यु किस वर्ष में हुई थी?,"२७ अक्तूबर, १६०५",१५५६,0.0


In [45]:
res.jaccard.mean()

0.7046592499537636

In [46]:
from string import punctuation

def postuning(s):
    s = " ".join(s.split())
    s = s.strip(punctuation)
    return s

res['postuned'] = res['prediction'].apply(postuning)

In [47]:
res['pjaccard'] = res[['answer', 'postuned']].apply(jaccard, axis=1)
res.pjaccard.mean()

0.7109129902290479